In [4]:
import os
from habnet.featuriser.featurise import Featuriser, MOL_TYPES
from habnet.featuriser.habnet_featurizer import AtomHAbNetFeaturizer
from chemprop import data, featurizers, nn
from chemprop.nn import metrics
from chemprop.models import multi
from chemprop.nn.metrics import MSE
from lightning import pytorch as pl
from chemprop.featurizers import MorganBinaryFeaturizer

In [ ]:
import ast
import pandas as pd

def find_matching_atoms_column(df, mol, rxn_name):
    num_atoms = mol.GetNumAtoms()
    subset = df[df['rxn'] == rxn_name]

    if subset.empty:
        raise ValueError(f"No rows found for rxn={rxn_name}")

    # Get all candidate columns that end in "atoms"
    atom_cols = [col for col in df.columns if col.endswith("atoms")]

    match_counts = {}

    for col in atom_cols:
        try:
            cell = subset.iloc[0][col]
            if pd.isna(cell):
                continue

            d = ast.literal_eval(cell)
            if not isinstance(d, dict):
                continue

            if len(d) == num_atoms:
                match_counts[col] = len(d)
        except Exception as e:
            print(f"[WARN] Column {col} failed with: {e}")

    # Safety: only allow one exact match
    if len(match_counts) == 0:
        raise ValueError(f"No column matched num_atoms={num_atoms} for rxn={rxn_name}")
    elif len(match_counts) > 1:
        raise ValueError(f"Ambiguous match: multiple columns matched num_atoms={num_atoms} → {match_counts}")

    # Success: return the only matching column
    atom_name = list(match_counts.keys())[0]
    # Remove the "_atoms" suffix and add "_bonds"
    bond_name = atom_name.replace("_atoms", "_bonds")
    mol_name = atom_name.replace("_atoms", "_mol")
    return atom_name, bond_name, mol_name



def get_bond_features(mol, data):
    """
    Get bond features for a molecule.
    """
    translation_bond = {}
    for bond in mol.GetBonds():
        atom1 = bond.GetBeginAtomIdx()
        atom2 = bond.GetEndAtomIdx()
        translation_bond[f"{atom1}-{atom2}"] = bond.GetIdx()
    # Change the key in data by using the key in translation_bond to get the value and make it the key
    new_data = {}
    for key, value in data.items():
        if key in translation_bond:
            new_data[translation_bond[key]] = value
        else:
            new_data[key] = value
    return new_data
    


In [5]:
feat_data = Featuriser(os.path.expanduser("~/code/chemprop_phd_customised/habnet/data/processed/sdf_data"), 
                       path = "/home/calvin/code/chemprop_phd_customised/habnet/data/processed/target_data/target_data_slim.csv",
                       include_extra_features = False)

Reaction rxn_191 not found in target data
Reaction rmg_rxn_16785 not found in target data
Reaction rxn_1169 not found in target data
Reaction rxn_144 not found in target data
Reaction rmg_rxn_16786 not found in target data
Reaction rxn_1257 not found in target data
Reaction rmg_rxn_1155 not found in target data
Reaction rmg_rxn_1160 not found in target data
Reaction rxn_798 not found in target data
Reaction rxn_682 not found in target data
Reaction rxn_30 not found in target data
Reaction rxn_795 not found in target data
Reaction rxn_111 not found in target data
Reaction rxn_635 not found in target data
Reaction rxn_824 not found in target data
Reaction rxn_182 not found in target data
Reaction rxn_89 not found in target data
Reaction rxn_127 not found in target data
Reaction rxn_286 not found in target data
Reaction rxn_262 not found in target data
Reaction rxn_792 not found in target data
Reaction rxn_909 not found in target data
Reaction rxn_791 not found in target data
Reaction rxn

In [27]:
import pandas as pd
df = pd.read_csv("/home/calvin/code/GINE/reaction_features.csv")

In [7]:
component_to_split_by = 0
mols = [d.mol for d in feat_data[component_to_split_by]]

train_indices, val_indices, test_indices = data.make_split_indices(mols, "kennard_stone", (0.8, 0.1, 0.1))


train_data, val_data, test_data = data.split_data_by_indices(
    feat_data, train_indices, val_indices, test_indices
)

The return type of make_split_indices has changed in v2.1 - see help(make_split_indices)


In [10]:
train_data[0][0][0]

MoleculeDatapoint(mol=<rdkit.Chem.rdchem.Mol object at 0x7b54c298d310>, y=[-0.2687813250856759, 0.9632012247112168], weight=1.0, gt_mask=None, lt_mask=None, x_d=None, x_phase=None, name='rmg_rxn_1563_r1h', V_f=None, E_f=None, V_d=None)

Index(['rxn', 'r0', 'r1', 'p0', 'p1'], dtype='object')

In [42]:
smiles_df = pd.read_csv("/home/calvin/code/chemprop_phd_customised/habnet/data/preprocessing/ordered_reactions_data/ordered_habstraction_smiles.csv")


import numpy as np
import json
col = "orig_" +train_data[0][0][0].name.split("_")[-1]
rxn = "_".join(train_data[0][0][0].name.split("_")[:-1] )

smiles_info = smiles_df[smiles_df["rxn"] == rxn][col].values[0]


bf_atoms = df[df["rxn"] == rxn][smiles_info+"_atoms"].values[0]
# parse the JSON string into a dict
bf_dict = json.loads(bf_atoms)

# sort keys as integers and build a list of rows
rows = [bf_dict[str(i)] for i in sorted(map(int, bf_dict.keys()))]

# convert to a 2D numpy array: each sub‐list is [q_mull, q_apt, spin, Z, mass]
atoms_array = np.array([[r["q_mull"], r["q_apt"], r["spin"], r["Z"], r["mass"]] for r in rows])

print(atoms_array)

bf_bonds = df[df["rxn"] == rxn][smiles_info+"_bonds"].values[0]
mol = train_data[0][0][0].mol

dict_bonds = {}
for bond in mol.GetBonds():
    # Find which atoms are connected by this bond
    atom1 = bond.GetBeginAtom()
    atom2 = bond.GetEndAtom()
    # make the key in the dict_bonds
    key = f"{atom1.GetIdx()}-{atom2.GetIdx()}"
    # the value is the bond idx
    dict_bonds[key] = bond.GetIdx()


# parse the JSON string into a dict
bf_dict = json.loads(bf_bonds)
# sort keys as integers and build a list of rows
rows = [bf_dict[str(i)] for i in sorted(map(int, bf_dict.keys()))]
# convert to a 2D numpy array: each sub‐list is [bond_type, bond_order, bond_length]
bonds_array = np.array([[r["bond_type"], r["bond_order"], r["bond_length"]] for r in rows])
print(bonds_array)


[[-0.28658   -0.523082   0.         7.        14.003074 ]
 [ 0.13002    0.643371   0.         7.        14.003074 ]
 [-0.345656  -0.559131   0.         8.        15.9949146]
 [ 0.237948   0.189013   0.         1.         1.007825 ]
 [ 0.264269   0.24983    0.         1.         1.007825 ]]


ValueError: invalid literal for int() with base 10: '0-1'

In [ ]:
df[df['rxn'] == rxn]


,rxn,r0_atoms,r0_bonds,r0_mol,r1_atoms,r1_bonds,r1_mol,p0_atoms,p0_bonds,p0_mol,p1_atoms,p1_bonds,p1_mol
70,rmg_rxn_1563,"{""0"": {""q_mull"": -0.28658, ""q_apt"": -0.523082,...","{""0-1"": {""length"": 1.3204199077111796, ""wbo"": ...","{""E_scf"": -5057.700580827224, ""dipole_mag"": 3....","{""0"": {""q_mull"": -0.28177, ""q_apt"": -0.1447, ""...","{""0-1"": {""length"": 1.321807534543513, ""wbo"": 0...","{""E_scf"": -8353.805573598622, ""dipole_mag"": 1....","{""0"": {""q_mull"": -0.273413, ""q_apt"": -0.437329...","{""0-1"": {""length"": 1.2322636460161438, ""wbo"": ...","{""E_scf"": -5040.323875431655, ""dipole_mag"": 2....","{""0"": {""q_mull"": -0.296508, ""q_apt"": -0.148559...","{""0-1"": {""length"": 1.32232515980677, ""wbo"": 0....","{""E_scf"": -8372.172902936203, ""dipole_mag"": 0...."


('p1_atoms', 'p1_bonds', 'p1_mol')

In [60]:
df[df["rxn"] == rxn]['p1_atoms'].values[0]

'{"0": {"q_mull": -0.296508, "q_apt": -0.148559, "spin": 0.0, "Z": 6, "mass": 12.0}, "1": {"q_mull": -0.167689, "q_apt": 0.004623, "spin": 0.0, "Z": 6, "mass": 12.0}, "2": {"q_mull": 0.005764, "q_apt": 0.445904, "spin": 0.0, "Z": 6, "mass": 12.0}, "3": {"q_mull": -0.338569, "q_apt": -0.801984, "spin": 0.0, "Z": 8, "mass": 15.9949146}, "4": {"q_mull": 0.111149, "q_apt": 0.914772, "spin": 0.0, "Z": 6, "mass": 12.0}, "5": {"q_mull": -0.469116, "q_apt": -0.655716, "spin": 0.0, "Z": 8, "mass": 15.9949146}, "6": {"q_mull": 0.141597, "q_apt": 0.05434, "spin": 0.0, "Z": 1, "mass": 1.007825}, "7": {"q_mull": 0.129203, "q_apt": 0.040584, "spin": 0.0, "Z": 1, "mass": 1.007825}, "8": {"q_mull": 0.118364, "q_apt": 0.012678, "spin": 0.0, "Z": 1, "mass": 1.007825}, "9": {"q_mull": 0.125652, "q_apt": -0.013335, "spin": 0.0, "Z": 1, "mass": 1.007825}, "10": {"q_mull": 0.116638, "q_apt": -0.025916, "spin": 0.0, "Z": 1, "mass": 1.007825}, "11": {"q_mull": 0.101614, "q_apt": -0.057707, "spin": 0.0, "Z": 1

In [56]:
mol = train_data[0][1][0].mol
mol.GetNumAtoms()

5

In [58]:
import ast
keys_num = len(ast.literal_eval(df[df["rxn"] == rxn]['r0_atoms'].values[0]).keys())
print(keys_num)

5


In [74]:


def get_bond_features(mol, data):
    """
    Map QM bond features (with keys like '0-1') to RDKit bond indices.
    Accepts reversed keys like '1-0' too.
    """
    translation_bond = {}
    for bond in mol.GetBonds():
        i = bond.GetBeginAtomIdx()
        j = bond.GetEndAtomIdx()
        translation_bond[f"{i}-{j}"] = bond.GetIdx()
        translation_bond[f"{j}-{i}"] = bond.GetIdx()  # <- add reverse

    new_data = {}
    for key, value in data.items():
        if key in translation_bond:
            bond_idx = translation_bond[key]
            new_data[bond_idx] = value
        else:
            print(f"[WARN] No bond match for key: {key}")
            # optionally: new_data[key] = value or skip

    return new_data
    
get_bond_features(train_data[0][0][0].mol, ast.literal_eval(df[df["rxn"] == rxn]['p1_bonds'].values[0]))

[WARN] No bond match for key: 0-1
[WARN] No bond match for key: 0-7
[WARN] No bond match for key: 2-10
[WARN] No bond match for key: 4-11
[WARN] No bond match for key: 4-12


{0: {'length': 1.0832395909686832, 'wbo': 0.0, 'k_phi': 0.0},
 1: {'length': 1.5003651252511836, 'wbo': 0.0, 'k_phi': 0.0},
 3: {'length': 1.08679145156235, 'wbo': 0.0, 'k_phi': 0.0},
 4: {'length': 1.4110371382337885, 'wbo': 0.0, 'k_phi': 0.0},
 5: {'length': 1.096311226671514, 'wbo': 0.0, 'k_phi': 0.0},
 6: {'length': 1.3972574560251954, 'wbo': 0.0, 'k_phi': 0.0},
 9: {'length': 1.3979753139694562, 'wbo': 0.0, 'k_phi': 0.0},
 12: {'length': 0.9597508552541124, 'wbo': 0.0, 'k_phi': 0.0}}

In [75]:
mol = train_data[0][0][0].mol
rxn = "_".join(train_data[0][0][0].name.split("_")[:-1])
df[df["rxn"] == rxn]


,rxn,r0_atoms,r0_bonds,r0_mol,r1_atoms,r1_bonds,r1_mol,p0_atoms,p0_bonds,p0_mol,p1_atoms,p1_bonds,p1_mol
70,rmg_rxn_1563,"{""0"": {""q_mull"": -0.28658, ""q_apt"": -0.523082,...","{""0-1"": {""length"": 1.3204199077111796, ""wbo"": ...","{""E_scf"": -5057.700580827224, ""dipole_mag"": 3....","{""0"": {""q_mull"": -0.28177, ""q_apt"": -0.1447, ""...","{""0-1"": {""length"": 1.321807534543513, ""wbo"": 0...","{""E_scf"": -8353.805573598622, ""dipole_mag"": 1....","{""0"": {""q_mull"": -0.273413, ""q_apt"": -0.437329...","{""0-1"": {""length"": 1.2322636460161438, ""wbo"": ...","{""E_scf"": -5040.323875431655, ""dipole_mag"": 2....","{""0"": {""q_mull"": -0.296508, ""q_apt"": -0.148559...","{""0-1"": {""length"": 1.32232515980677, ""wbo"": 0....","{""E_scf"": -8372.172902936203, ""dipole_mag"": 0...."


In [69]:
from rdkit.Chem import SanitizeMol
SanitizeMol(mol, catchErrors=True)

rdkit.Chem.rdmolops.SanitizeFlags.SANITIZE_NONE

In [76]:
ast.literal_eval(df[df["rxn"] == rxn]['p1_bonds'].values[0])

{'0-1': {'length': 1.32232515980677, 'wbo': 0.0, 'k_phi': 0.0},
 '0-6': {'length': 1.0832395909686832, 'wbo': 0.0, 'k_phi': 0.0},
 '0-7': {'length': 1.0821290646591097, 'wbo': 0.0, 'k_phi': 0.0},
 '1-2': {'length': 1.5003651252511836, 'wbo': 0.0, 'k_phi': 0.0},
 '1-8': {'length': 1.08679145156235, 'wbo': 0.0, 'k_phi': 0.0},
 '2-3': {'length': 1.4110371382337885, 'wbo': 0.0, 'k_phi': 0.0},
 '2-9': {'length': 1.096311226671514, 'wbo': 0.0, 'k_phi': 0.0},
 '2-10': {'length': 1.0935341385128312, 'wbo': 0.0, 'k_phi': 0.0},
 '3-4': {'length': 1.3972574560251954, 'wbo': 0.0, 'k_phi': 0.0},
 '4-5': {'length': 1.3979753139694562, 'wbo': 0.0, 'k_phi': 0.0},
 '4-11': {'length': 1.0934813046531704, 'wbo': 0.0, 'k_phi': 0.0},
 '4-12': {'length': 1.0935047971586591, 'wbo': 0.0, 'k_phi': 0.0},
 '5-13': {'length': 0.9597508552541124, 'wbo': 0.0, 'k_phi': 0.0}}

In [77]:
ast.literal_eval(df[df["rxn"] == rxn]['p1_atoms'].values[0])

{'0': {'q_mull': -0.296508,
  'q_apt': -0.148559,
  'spin': 0.0,
  'Z': 6,
  'mass': 12.0},
 '1': {'q_mull': -0.167689,
  'q_apt': 0.004623,
  'spin': 0.0,
  'Z': 6,
  'mass': 12.0},
 '2': {'q_mull': 0.005764,
  'q_apt': 0.445904,
  'spin': 0.0,
  'Z': 6,
  'mass': 12.0},
 '3': {'q_mull': -0.338569,
  'q_apt': -0.801984,
  'spin': 0.0,
  'Z': 8,
  'mass': 15.9949146},
 '4': {'q_mull': 0.111149,
  'q_apt': 0.914772,
  'spin': 0.0,
  'Z': 6,
  'mass': 12.0},
 '5': {'q_mull': -0.469116,
  'q_apt': -0.655716,
  'spin': 0.0,
  'Z': 8,
  'mass': 15.9949146},
 '6': {'q_mull': 0.141597,
  'q_apt': 0.05434,
  'spin': 0.0,
  'Z': 1,
  'mass': 1.007825},
 '7': {'q_mull': 0.129203,
  'q_apt': 0.040584,
  'spin': 0.0,
  'Z': 1,
  'mass': 1.007825},
 '8': {'q_mull': 0.118364,
  'q_apt': 0.012678,
  'spin': 0.0,
  'Z': 1,
  'mass': 1.007825},
 '9': {'q_mull': 0.125652,
  'q_apt': -0.013335,
  'spin': 0.0,
  'Z': 1,
  'mass': 1.007825},
 '10': {'q_mull': 0.116638,
  'q_apt': -0.025916,
  'spin': 0.0,


In [144]:
file_to_read = "/home/calvin/Dropbox/PersonalFolders/Calvin/ATLAS_Converged/rmg_rxn_1563/calcs/Species/rmg_rxn_1563_p2_C=CCOCO/opt_a36160/input.log"
def extract_final_xyz_archive_block(logfile: str) -> tuple[list[str], np.ndarray]:
    """
    Extracts atomic symbols and XYZ coordinates from the final Gaussian archive block.
    Works even when the archive starts with job info before \\0,1\.
    """
    import numpy as np

    with open(logfile, "r") as f:
        lines = f.readlines()

    archive_lines = []
    recording = False

    for line in reversed(lines):
        line = line.strip()
        if "\\0,1\\C," in line or "\\0,1\\H," in line:
            recording = True
            archive_lines.append(line)
        elif recording and line.startswith("\\"):
            archive_lines.append(line)
        elif recording:
            break

    if not archive_lines:
        raise ValueError("Could not find Gaussian archive coordinate block.")

    # Combine and clean up
    archive = "".join(reversed(archive_lines))
    # Strip everything before \0,1
    if "\\0,1" in archive:
        archive = archive.split("\\0,1", 1)[-1]

    entries = archive.strip("\\").split("\\")

    symbols = []
    coords = []

    for entry in entries:
        fields = entry.split(",")
        if len(fields) != 4:
            continue
        sym, x, y, z = fields
        try:
            symbols.append(sym)
            coords.append([float(x), float(y), float(z)])
        except ValueError:
            continue

    return symbols, np.array(coords, dtype=np.float64)


extract_final_xyz_archive_block(file_to_read)

<>:3: SyntaxWarning: invalid escape sequence '\.'
<>:3: SyntaxWarning: invalid escape sequence '\.'
/tmp/ipykernel_2935604/1093345666.py:3: SyntaxWarning: invalid escape sequence '\.'
  """


(['C'], array([[-2.6591063 , -0.04495296,  0.70003   ]]))

In [145]:
file_to_read = "/home/calvin/Dropbox/PersonalFolders/Calvin/ATLAS_Converged/rmg_rxn_1563/calcs/Species/rmg_rxn_1563_p2_C=CCOCO/opt_a36160/"
t = os.listdir(file_to_read)


In [146]:
# print the last 200 lines of lines
for line in lines[-200:]:
    print(line)

 ---------------------------------------------------------------------

 Rotational constants (GHZ):      8.0892305      2.0459128      1.8918708

 Leave Link  202 at Sun Feb 23 12:12:31 2025, MaxMem=   805306368 cpu:         0.0

 (Enter /Local/ce_dana/g09/l601.exe)

 Copying SCF densities to generalized density rwf, IOpCl= 0 IROHF=0.



 **********************************************************************



            Population analysis using the SCF density.



 **********************************************************************



 Orbital symmetries:

       Occupied  (A) (A) (A) (A) (A) (A) (A) (A) (A) (A) (A) (A)

                 (A) (A) (A) (A) (A) (A) (A) (A) (A) (A) (A) (A)

       Virtual   (A) (A) (A) (A) (A) (A) (A) (A) (A) (A) (A) (A)

                 (A) (A) (A) (A) (A) (A) (A) (A) (A) (A) (A) (A)

                 (A) (A) (A) (A) (A) (A) (A) (A) (A) (A) (A) (A)

                 (A) (A) (A) (A) (A) (A) (A) (A) (A) (A) (A) (A)

                 (A) (A) (A) (A) 

In [147]:
# Find where 'Test jon not archived.' in the lines
for i, line in enumerate(lines):
    if "Test job not archived." in line:
        print(i)
        break

i+1
print(lines[i+1])
# Find the next line that contains '\\0,'
for j, line in enumerate(lines[i+1:]):
    if "\\0," in line:
        print(i+j+1)
        break


1912
 1\1\GINC-TECH-WN095\FOpt\RwB97XD\def2TZVP\C4H8O2\CALVINP\23-Feb-2025\0

1916


In [148]:
lines[i+j+1]

' t)\\\\rmg_rxn_1563_p2_C=CCOCO\\\\0,1\\C,-2.6591062964,-0.0449529599,0.70003\n'

In [151]:
def extract_xyz_after_test_not_archived(logfile: str) -> tuple[list[str], np.ndarray]:
    """
    Extracts XYZ coordinates from Gaussian log file, starting after 'Test job not archived.'
    and stopping before '\\Version='. Returns (symbols, coords).
    """
    import numpy as np

    with open(logfile, "r") as f:
        lines = f.readlines()

    start_idx = None
    for i, line in enumerate(lines):
        if "Test job not archived." in line:
            start_idx = i + 1
            break

    if start_idx is None:
        raise ValueError("Could not find 'Test job not archived.' in log.")

    # Find the \\0, line
    coord_start = None
    for j, line in enumerate(lines[start_idx:]):
        if "\\0," in line:
            coord_start = start_idx + j
            break

    if coord_start is None:
        raise ValueError("Could not find line with '\\0,' after job archive.")

    # Read lines from coord_start until we hit '\Version='
    coord_lines = []
    for line in lines[coord_start:]:
        if "\\Version=" in line:
            break
        coord_lines.append(line.strip())

    full = "".join(coord_lines)

    # Now we have something like: \0,1\C,x,y,z\C,x,y,z...\H,x,y,z\...\H,x,y,z
    # Strip prefix and split into atomic entries
    if "\\0,1" in full:
        full = full.split("\\0,1", 1)[-1]

    tokens = full.strip("\\").split("\\")
    symbols = []
    coords = []

    for token in tokens:
        parts = token.split(",")
        if len(parts) != 4:
            continue
        sym, x, y, z = parts
        try:
            symbols.append(sym)
            coords.append([float(x), float(y), float(z)])
        except ValueError:
            print(f"[WARN] Could not parse: {token}")
            continue

    return symbols, np.array(coords, dtype=np.float64)

qm = extract_xyz_after_test_not_archived(file_to_read + "input.log")

In [152]:
qm

(['C', 'C', 'C', 'O', 'C', 'O', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H'],
 array([[-2.6591063 , -0.04495296,  0.70003096],
        [-1.37366879,  0.04991392,  0.99532386],
        [-0.24085047, -0.37508605,  0.10808613],
        [-0.65405618, -1.02530529, -1.07407397],
        [-0.85560979, -2.40028858, -0.92873241],
        [ 0.33932063, -3.11251714, -0.79014811],
        [-2.98906438, -0.45504203, -0.24673331],
        [-3.41977321,  0.29099743,  1.39250923],
        [-1.06687856,  0.47386522,  1.94782583],
        [ 0.45350224, -1.01168879,  0.66889817],
        [ 0.32474663,  0.50784315, -0.20231896],
        [-1.40786164, -2.70616604, -1.82156954],
        [-1.43878786, -2.62655132, -0.03181419],
        [ 0.85426668, -2.99214853, -1.59106271]]))

In [150]:
from scipy.spatial import cKDTree

from scipy.spatial import cKDTree
import numpy as np

def match_qm_to_rdkit_robust(mol, qm_xyz, qm_symbols, tol=0.3):
    """
    Matches QM atoms to RDKit atoms by atomic symbol and 3D distance.
    Returns: mapping[i] = j → QM atom i maps to RDKit atom j
    """
    from scipy.spatial.distance import cdist
    import numpy as np

    rdkit_xyz = np.array([list(mol.GetConformer().GetAtomPosition(i)) for i in range(mol.GetNumAtoms())])
    rdkit_symbols = np.array([a.GetSymbol() for a in mol.GetAtoms()])
    mapping = np.full(len(qm_xyz), -1, dtype=int)

    used_rdkit = set()

    for i, (sym_qm, coord_qm) in enumerate(zip(qm_symbols, qm_xyz)):
        # Candidates: RDKit atoms with same symbol not already used
        candidates = np.where(rdkit_symbols == sym_qm)[0]
        candidates = [j for j in candidates if j not in used_rdkit]

        if not candidates:
            raise RuntimeError(f"No unused RDKit atom found for QM atom {i} ({sym_qm})")

        dists = cdist([coord_qm], rdkit_xyz[candidates])[0]
        min_j = candidates[np.argmin(dists)]
        if dists.min() > tol:
            raise RuntimeError(f"QM atom {i} ({sym_qm}) too far from RDKit candidate (min dist={dists.min():.2f})")

        mapping[i] = min_j
        used_rdkit.add(min_j)

    return mapping



mapping = match_qm_to_rdkit_robust(mol, qm[1], qm[0])

RuntimeError: QM atom 0 (C) too far from RDKit candidate (min dist=1.65)

In [141]:
for i, a in enumerate(mol.GetAtoms()):
    pos = mol.GetConformer().GetAtomPosition(i)
    print(f"RDKit atom {i:2d} ({a.GetSymbol()}): {pos.x:.2f} {pos.y:.2f} {pos.z:.2f}")

for i, (sym, coord) in enumerate(zip(qm[0], qm[1])):
    print(f"QM atom    {i:2d} ({sym}): {coord[0]:.2f} {coord[1]:.2f} {coord[2]:.2f}")


RDKit atom  0 (H): 0.74 -2.48 -0.65
RDKit atom  1 (C): -0.19 2.29 0.39
RDKit atom  2 (C): -1.13 1.43 0.05
RDKit atom  3 (C): -1.07 -0.06 0.26
RDKit atom  4 (O): 0.17 -0.52 0.74
RDKit atom  5 (C): 1.12 -0.76 -0.26
RDKit atom  6 (O): 0.83 -1.81 -1.07
RDKit atom  7 (H): -0.31 3.35 0.21
RDKit atom  8 (H): 0.73 1.96 0.86
RDKit atom  9 (H): -2.05 1.78 -0.41
RDKit atom 10 (H): -1.33 -0.57 -0.67
RDKit atom 11 (H): -1.82 -0.35 1.00
RDKit atom 12 (H): 2.08 -0.89 0.26
RDKit atom 13 (H): 1.22 0.12 -0.93
QM atom     0 (C): -2.66 -0.04 0.70
QM atom     1 (C): -1.37 0.05 1.00
QM atom     2 (C): -0.24 -0.38 0.11
QM atom     3 (O): -0.65 -1.03 -1.07
QM atom     4 (C): -0.86 -2.40 -0.93
QM atom     5 (O): 0.34 -3.11 -0.79
QM atom     6 (H): -2.99 -0.46 -0.25
QM atom     7 (H): -3.42 0.29 1.39
QM atom     8 (H): -1.07 0.47 1.95
QM atom     9 (H): 0.45 -1.01 0.67
QM atom    10 (H): 0.32 0.51 -0.20
QM atom    11 (H): -1.41 -2.71 -1.82
QM atom    12 (H): -1.44 -2.63 -0.03
QM atom    13 (H): 0.85 -2.99 -1.59

In [143]:
from rdkit.Chem import rdMolAlign
from rdkit import Chem
from rdkit.Chem import AllChem, rdMolAlign

def assign_qm_xyz_to_mol(mol, qm_xyz):
    mol = Chem.Mol(mol)  # copy to avoid mutating original
    conf = mol.GetConformer()
    for i in range(mol.GetNumAtoms()):
        x, y, z = qm_xyz[i]
        conf.SetAtomPosition(i, (float(x), float(y), float(z)))
    return mol

assigned_mol = assign_qm_xyz_to_mol(mol, qm[1])
mol_qm_geom = assign_qm_xyz_to_mol(mol, qm[1])
rmsd = rdMolAlign.GetBestRMS(mol_qm_geom, mol)
print(f"RMSD: {rmsd:.3f} Å")

RMSD: 2.149 Å
